In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
import os
from langchain_community.llms import HuggingFaceHub   # only this for HF API


In [ ]:
load_dotenv()

from huggingface_hub import InferenceClient

client = InferenceClient("google/gemma-2-2b-it", token="paste-huggingface-token-here")


In [ ]:
class BlogState(TypedDict):
    title:str 
    outline: str 
    content: str 

In [ ]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']

    messages = [
        {"role": "system", "content": "You are an expert blog writer."},
        {"role": "user", "content": f"Generate a detailed outline for a blog on the topic: {title}"}
    ]

    response = client.chat.completions.create(
        messages=messages,
        max_tokens=50,
        temperature=0.7
    )

    state['outline'] = response.choices[0].message["content"]
    return state


def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']

    messages = [
        {"role": "system", "content": "You are an expert blog writer."},
        {"role": "user", "content": f"Write a detailed blog on the title '{title}' using the following outline:\n{outline}"}
    ]

    response = client.chat.completions.create(
        messages=messages,
        max_tokens=70,
        temperature=0.7
    )

    state['content'] = response.choices[0].message["content"]
    return state


In [ ]:
graph = StateGraph(BlogState)
#nodes 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

#edges 
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [ ]:
initial_state = {'title': 'Rise of AI in india'}
final_state = workflow.invoke(initial_state)

print(final_state)

In [ ]:
print(final_state['outline'])

In [ ]:
print(final_state['content'])